In [13]:
from datasets import load_from_disk

kr_jp = load_from_disk('../data/kr_jp_aihub')
kr_cn = load_from_disk('../data/kr_cn_aihub')

In [14]:
pd_jp_train = kr_jp['train'].to_pandas()
pd_jp_validation = kr_jp['validation'].to_pandas()
pd_cn_train = kr_cn['train'].to_pandas()
pd_cn_validation = kr_cn['validation'].to_pandas()

In [15]:
pd_jp_train.insert(0, 'lang', 'jp')
pd_cn_train.insert(0, 'lang', 'cn')
pd_jp_validation.insert(0, 'lang', 'jp')
pd_cn_validation.insert(0, 'lang', 'cn')

In [16]:
pd_train = pd_jp_train.append(pd_cn_train)
pd_validation = pd_jp_validation.append(pd_cn_validation)

In [17]:
pd_train['text'] = pd_train['text'].apply(lambda x: x.replace(' ', ''))
pd_validation['text'] = pd_validation['text'].apply(lambda x: x.replace(' ', ''))

In [18]:
pd_train = pd_train[pd_train.duplicated(subset=['text'], keep=False)]
pd_validation = pd_validation[pd_validation.duplicated(subset=['text'], keep=False)]

In [19]:
from tqdm import tqdm

train_dict = {}

for idx, row in tqdm(pd_train.iterrows()):
  if row['text'] not in train_dict.keys():
    train_dict[row['text']] = {}
  train_dict[row['text']][row['lang']] = row['target']

1675662it [01:40, 16691.24it/s]


In [20]:
from tqdm import tqdm

validation_dict = {}

for idx, row in tqdm(pd_validation.iterrows()):
  if row['text'] not in validation_dict.keys():
    validation_dict[row['text']] = {}
  validation_dict[row['text']][row['lang']] = row['target']

26322it [00:01, 16814.90it/s]


In [22]:
del_train = []
del_validation = []
for key in train_dict.keys():
  if len(train_dict[key]) != 2:
    del_train.append(key)
for key in validation_dict.keys():
  if len(validation_dict[key]) != 2:
    del_validation.append(key)
for key in del_train:
  del train_dict[key]
for key in del_validation:
  del validation_dict[key]

In [23]:
len(train_dict.keys())

837828

In [28]:
train = {
  'id': [],
  'title': [],
  'text': [],
  'target': []
}

validation = {
  'id': [],
  'title': [],
  'text': [],
  'target': []
}

for idx, key in enumerate(train_dict.keys()):
  train['id'].append(str(idx))
  train['title'].append('aihub')
  train['text'].append(train_dict[key]['jp'])
  train['target'].append(train_dict[key]['cn'])

for idx, key in enumerate(validation_dict.keys()):
  validation['id'].append(str(idx))
  validation['title'].append('aihub')
  validation['text'].append(validation_dict[key]['jp'])
  validation['target'].append(validation_dict[key]['cn'])

In [29]:
from datasets import Dataset, DatasetDict

train_datasets = Dataset.from_dict(train)
validation_datasets = Dataset.from_dict(validation)

In [30]:
jp_cn_datasets = DatasetDict({
  'train': train_datasets,
  'validation': validation_datasets
})

In [32]:
jp_cn_datasets.save_to_disk('../data/jp_cn_aihub')